In [ ]:
#!pip install autogluon

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
file_path = '/content/drive/My Drive/ProyectoTesis/AllFeatureData.pkl'


In [ ]:
import pandas as pd
import tensorflow as tf
import numpy as np

In [ ]:
allData = pd.read_pickle(file_path)

In [ ]:
featureData = allData.copy()

In [ ]:
features = allData.columns[:-1]
features

In [ ]:
for feature in features:
  featureData[feature] = featureData[feature].apply(lambda x: np.mean(x))

featureData.head()

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import classification_report, confusion_matrix

In [ ]:
X = featureData.drop('Label', axis=1)
y = featureData['Label']

In [ ]:
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

In [ ]:
featureData['Label'] = y_encoded

In [ ]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [ ]:
train_data, test_data = train_test_split(featureData, test_size=0.2, random_state=42, stratify=y)


In [ ]:
from autogluon.tabular import TabularPredictor

# Set the name of the target column
label = 'Label'

predictor = TabularPredictor(label=label, problem_type='binary', path="/content/drive/My Drive/ProyectoTesis/Autogluon/Models/", verbosity=2, eval_metric='f1-weighted')

In [ ]:
predictor.fit(
    train_data=train_data,
    time_limit=10800,  # Set training time limit (in seconds)
    presets='best_quality',  # Use the "best quality" preset to maximize performance
    ag_args_fit={'use_gpu': True},
    verbosity=2,
    excluded_model_types=['NN', 'FastAI'],
    dynamic_stacking=False
)

In [ ]:
import pickle as pkl

In [ ]:
with open(f'/content/drive/My Drive/ProyectoTesis/autogluon_predictor.pkl', 'wb') as file:
    pkl.dump(predictor, file)